In [1]:
from typing import List, Tuple, Union
import numpy as np
import torch
from monai.data import DataLoader, Dataset, CacheDataset, decollate_batch
import os
from monai.transforms import (
    Compose, 
    EnsureChannelFirstd, 
    Orientationd,  
    AsDiscrete,  
    RandFlipd, 
    RandRotate90d, 
    NormalizeIntensityd,
    RandCropByLabelClassesd,
)

In [2]:
TRAIN_IMG_DIR = "./datasets/train/images"
TRAIN_LABEL_DIR = "./datasets/train/labels"
VAL_IMG_DIR = "./datasets/val/images"
VAL_LABEL_DIR = "./datasets/val/labels"

train_list = os.listdir(TRAIN_IMG_DIR)
val_list = os.listdir(VAL_IMG_DIR)
train_files = []
valid_files = []


for name in train_list:
    train_image = np.load(os.path.join(TRAIN_IMG_DIR, f"{name}"))    
    train_label = np.load(os.path.join(TRAIN_LABEL_DIR, f"{name.replace("image", "label")}"))

    train_files.append({"image": train_image, "label": train_label})    

for name in val_list:
    valid_image = np.load(os.path.join(VAL_IMG_DIR, f"{name}"))
    valid_label = np.load(os.path.join(VAL_LABEL_DIR, f"{name.replace("image", "label")}"))

    valid_files.append({"image": valid_image, "label": valid_label})

In [3]:
# Non-random transforms to be cached
non_random_transforms = Compose([
    EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
    NormalizeIntensityd(keys="image"),
    Orientationd(keys=["image", "label"], axcodes="ASR")
])

raw_train_ds = CacheDataset(data=train_files, transform=non_random_transforms, cache_rate=1.0)


my_num_samples = 1
train_batch_size = 1

xy_patch = 96
z_patch = 3
# Random transforms to be applied during training
random_transforms = Compose([
    RandCropByLabelClassesd(
        keys=["image", "label"],
        label_key="label",
        spatial_size=[z_patch, xy_patch, xy_patch],
        num_classes=7,
        num_samples=my_num_samples
    ),
    RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[0, 2]),
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),    
])


train_ds = Dataset(data=raw_train_ds, transform=random_transforms)


# DataLoader remains the same
train_loader = DataLoader(
    train_ds,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=torch.cuda.is_available()
)


Loading dataset: 100%|██████████| 24/24 [00:09<00:00,  2.48it/s]


In [4]:
# 데이터 검증 함수
def inspect_batch(loader):
    # 첫 번째 배치 가져오기
    batch = next(iter(loader))
    
    print("=== 배치 데이터 검증 ===")
    print(f"이미지 shape: {batch['image'].shape}")
    print(f"이미지 dtype: {batch['image'].dtype}")
    print(f"이미지 값 범위: [{batch['image'].min():.3f}, {batch['image'].max():.3f}]")
    print("\n")
    print(f"라벨 shape: {batch['label'].shape}")
    print(f"라벨 dtype: {batch['label'].dtype}")
    print(f"라벨 고유값: {torch.unique(batch['label'])}")

# 실행
inspect_batch(train_loader)

=== 배치 데이터 검증 ===
이미지 shape: torch.Size([1, 1, 96, 96, 3])
이미지 dtype: torch.float32
이미지 값 범위: [-8.709, 3.110]


라벨 shape: torch.Size([1, 1, 96, 96, 3])
라벨 dtype: torch.uint8
라벨 고유값: tensor([0, 3], dtype=torch.uint8)


# Validation DataLoader

In [5]:
import os
import numpy as np
import json
from pathlib import Path

valid_transforms = Compose([
    EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
    NormalizeIntensityd(keys="image")
])

def load_validation_patches(patch_dir):
    """
    저장된 validation 패치들을 로드하여 데이터셋용 리스트 생성
    Args:
        patch_dir: 패치가 저장된 디렉토리 (images, labels 서브디렉토리와 coordinates.json 포함)
    """
    patch_dir = Path(patch_dir)
    val_patched_data = []
    
    # coordinates.json 로드
    with open(patch_dir / "coordinates.json", 'r') as f:
        coordinates = json.load(f)
    
    # 각 패치에 대해
    for coord in coordinates:
        image = np.load(patch_dir / "images" / coord["patch_file"])
        label = np.load(patch_dir / "labels" / coord["patch_file"])
        
        val_patched_data.append({
            "image": image,      # shape: (11, 96, 96)
            "label": label,      # shape: (96, 96)
            "coords": coord      # 원본 위치 정보 (옵션)
        })
    
    return val_patched_data

# 패치 데이터 로드
val_patched_data = load_validation_patches("./datasets/val_patches")

# Dataset과 DataLoader 설정
valid_ds = CacheDataset(data=val_patched_data, transform=valid_transforms, cache_rate=1.0)

valid_batch_size = 1
valid_loader = DataLoader(
    valid_ds,
    batch_size=valid_batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=torch.cuda.is_available()
)


Loading dataset: 100%|██████████| 26496/26496 [00:07<00:00, 3439.67it/s]


In [6]:
# 데이터 검증 함수
def inspect_batch(loader):
    # 첫 번째 배치 가져오기
    batch = next(iter(loader))
    
    print("=== 배치 데이터 검증 ===")
    print(f"이미지 shape: {batch['image'].shape}")
    print(f"이미지 dtype: {batch['image'].dtype}")
    print(f"이미지 값 범위: [{batch['image'].min():.3f}, {batch['image'].max():.3f}]")
    print("\n")
    print(f"라벨 shape: {batch['label'].shape}")
    print(f"라벨 dtype: {batch['label'].dtype}")
    print(f"라벨 고유값: {torch.unique(batch['label'])}")

# 실행
inspect_batch(valid_loader)

=== 배치 데이터 검증 ===
이미지 shape: torch.Size([1, 1, 3, 96, 96])
이미지 dtype: torch.float32
이미지 값 범위: [-4.900, 5.728]


라벨 shape: torch.Size([1, 1, 96, 96])
라벨 dtype: torch.uint8
라벨 고유값: tensor([0, 5], dtype=torch.uint8)


In [7]:
import torch
import torch.nn as nn
from monai.networks.nets import UNet

class UNet2_5D_v2(nn.Module):
    def __init__(self, out_channels=6):
        super().__init__()
        
        # 초기 3D 처리 레이어
        self.init_3d = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=(11, 3, 3), padding=(0, 1, 1)),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )
        
        # 2D UNet
        self.unet = UNet(
            spatial_dims=2,
            in_channels=64,  # 3D 컨볼루션 출력 채널
            out_channels=out_channels,
            channels=(128, 256, 512, 1024),
            strides=(2, 2, 2, 2),
            num_res_units=2
        )

    def forward(self, x):
        # x shape: (batch, 1, 11, H, W)
        # 3D 처리
        x = x.unsqueeze(1)
        x = self.init_3d(x)  # (batch, 64, 1, H, W)
        x = x.squeeze(2)     # (batch, 64, H, W)
        
        # 2D UNet
        return self.unet(x)

# 테스트 코드

model = UNet2_5D_v2(out_channels=7)
x = torch.randn(2, 11, 224, 224)
output = model(x)
print(f"Output shape: {output.shape}")  # Expected: (8, 6, 256, 256)

/Users/seungwoo/anaconda3/envs/dust/lib/python3.12/site-packages/monai/networks/nets/unet.py:130: UserWarning: `len(strides) > len(channels) - 1`, the last 1 values of strides will not be used.
  warnings.warn(f"`len(strides) > len(channels) - 1`, the last {delta} values of strides will not be used.")


Output shape: torch.Size([2, 7, 224, 224])


In [9]:
for inputs, targets in valid_loader:
    print(f"Max label value: {targets.max().item()}")
    print(f"Num classes (model output): {output.shape[1]}")
    break

ValueError: too many values to unpack (expected 2)

In [ ]:
import torch
from monai.losses import DiceLoss
from torch import optim

# Loss and Optimizer
criterion = DiceLoss(to_onehot_y=True, softmax=True)  # Dice Loss with softmax
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in dataloader:
        # Ensure targets are long integers
        targets = targets.long()  # 라벨: (B, H, W)

        # Add channel dimension to targets: (B, H, W) -> (B, 1, H, W)
        targets = targets.unsqueeze(1)
        print(f"Modified Targets shape: {targets.shape}")  # (B, 1, H, W)

        # Forward pass
        outputs = model(inputs)  # 모델 출력: (B, 7, H, W)
        print(f"Outputs shape: {outputs.shape}, Targets shape: {targets.shape}")

        # Compute loss
        loss = criterion(outputs, targets)  # Dice Loss
        print(f"Loss: {loss.item()}")

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch loss
    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.9693675637245178
Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.9682437181472778
Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.9593340754508972
Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.9628776907920837
Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.957891047000885
Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.9588175415992737